In [1]:
pip install python-chess

In [2]:
import chess.pgn
import chess.engine
import multiprocessing
import io

def analyze_single_game(game_data):
    game_text, engine_path = game_data
    # Parse the game from PGN text
    game = chess.pgn.read_game(io.StringIO(game_text))
    if game is None:
        return None  # Handle empty or invalid games

    board = game.board()

    # Load the chess engine
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)
    # Configure the engine
    engine.configure({'Threads': 1, 'Hash': 128})

    # Extract Elo ratings and result
    white_elo = int(game.headers.get("WhiteElo", "0"))
    black_elo = int(game.headers.get("BlackElo", "0"))
    result = game.headers.get("Result", "*")

    move_number = 1
    evaluations = []

    # Go through each move
    for move in game.mainline_moves():
        # Get the SAN notation before pushing the move
        move_san = board.san(move)

        # Apply the move to the board
        board.push(move)

        # Get Stockfish evaluation with depth limit
        info = engine.analyse(board, chess.engine.Limit(depth=12))
        eval_score = info["score"].relative

        if eval_score.is_mate():
            eval_value = f"Mate in {eval_score.mate()}"
        else:
            eval_value = eval_score.score() / 100  # Convert centipawns to pawns

        # Collect the move and evaluation
        evaluations.append({
            'move_number': move_number,
            'move': move_san,
            'evaluation': eval_value
        })

        move_number += 1

    engine.quit()

    return {
        'white_elo': white_elo,
        'black_elo': black_elo,
        'result': result,
        'evaluations': evaluations
    }

def read_games_from_pgn(pgn_file):
    games = []
    with open(pgn_file) as pgn:
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:
                break
            # Convert the game back to PGN text
            exporter = chess.pgn.StringExporter(headers=True, variations=False, comments=False)
            game_text = game.accept(exporter)
            games.append(game_text)
    return games

def analyze_pgn_file(pgn_file, engine_path):
    games_text = read_games_from_pgn(pgn_file)
    game_data = (games_text[0], engine_path)
    result = analyze_single_game(game_data)
    return [result]

if __name__ == '__main__':
    # Paths to the Stockfish engine and PGN file
    stockfish_path = r"C:\Users\foivo\Downloads\stockfish-windows-x86-64-avx2\stockfish\stockfish-windows-x86-64-avx2.exe"
    pgn_file = "example.pgn"

    # Analyze the PGN file
    games_analysis = analyze_pgn_file(pgn_file, stockfish_path)

    # Example: Print the analysis of the first game
    if games_analysis:
        first_game = games_analysis[0]
        if first_game is not None:
            print(f"White Elo: {first_game['white_elo']}, Black Elo: {first_game['black_elo']}, Result: {first_game['result']}")
            for move_eval in first_game['evaluations']:
                print(f"Move {move_eval['move_number']}: {move_eval['move']}, Evaluation: {move_eval['evaluation']}")
        else:
            print("First game analysis returned None.")


White Elo: 2755, Black Elo: 2733, Result: 1/2-1/2
Move 1: c4, Evaluation: -0.03
Move 2: c6, Evaluation: 0.57
Move 3: Nf3, Evaluation: -0.27
Move 4: d5, Evaluation: 0.31
Move 5: e3, Evaluation: -0.36
Move 6: Nf6, Evaluation: 0.36
Move 7: Nc3, Evaluation: -0.3
Move 8: Bf5, Evaluation: 0.26
Move 9: Qb3, Evaluation: -0.09
Move 10: Qb6, Evaluation: 0.12
Move 11: cxd5, Evaluation: 0.02
Move 12: Qxb3, Evaluation: -0.01
Move 13: axb3, Evaluation: 0.02
Move 14: Nxd5, Evaluation: -0.02
Move 15: Nd4, Evaluation: 1.05
Move 16: Nb4, Evaluation: -1.28
Move 17: Kd1, Evaluation: 1.37
Move 18: Bg6, Evaluation: -1.07
Move 19: e4, Evaluation: 1.39
Move 20: Nd7, Evaluation: -1.29
Move 21: Nc2, Evaluation: 1.27
Move 22: e5, Evaluation: -1.21
Move 23: f3, Evaluation: 1.51
Move 24: f6, Evaluation: -1.4
Move 25: Nxb4, Evaluation: 1.55
Move 26: Bxb4, Evaluation: -1.7
Move 27: Kc2, Evaluation: 1.67
Move 28: Ke7, Evaluation: -1.42
Move 29: Ne2, Evaluation: 1.15
Move 30: Nc5, Evaluation: -0.79
Move 31: d4, Evalua